In [6]:
import torch
from torch import nn 
import numpy as np
import gym
import matplotlib.pyplot as plt
import time

In [7]:
class CEM(nn.Module):
    def __init__(self, state_dim, action_n, lr=0.01):
        super().__init__()
        self.state_dim = state_dim
        self.action_n = action_n
        
        self.network = nn.Sequential(
            nn.Linear(self.state_dim, 32), 
            nn.ReLU(), 
            # nn.Linear(64, 256), 
            # nn.ReLU(), 
            nn.Linear(32, self.action_n)
        )
        
        self.softmax = nn.Softmax()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, _input):
        return self.network(_input) 
    
    def get_action(self, state):
        state = torch.FloatTensor(state)
        logits = self.forward(state)
        action_prob = self.softmax(logits).detach().numpy()
        action = np.random.choice(self.action_n, p=action_prob)
        return action
    
    def update_policy(self, elite_trajectories):
        elite_states = []
        elite_actions = []
        for trajectory in elite_trajectories:
            elite_states.extend(trajectory['states'])
            elite_actions.extend(trajectory['actions'])

        elite_states = np.array(elite_states)
        elite_actions = np.array(elite_actions)

        elite_states_tensor = torch.FloatTensor(elite_states)
        elite_actions_tensor = torch.LongTensor(elite_actions)

        loss = self.loss(self.forward(elite_states_tensor), elite_actions_tensor)
        loss.backward()
        self.optimizer.step()

        self.optimizer.zero_grad()

In [8]:
def get_trajectory(env, agent, trajectory_len, visualize=False):
    trajectory = {'states':[], 'actions': [], 'total_reward': 0}
    
    state = env.reset()
    
    for _ in range(trajectory_len):
        trajectory['states'].append(state)
        action = agent.get_action(state)
        trajectory['actions'].append(action)
        state, reward, done, _ = env.step(action)
        trajectory['total_reward'] += reward

        if done:
            break
        if visualize:
            env.render()
            time.sleep(0.01)
                        
    return trajectory

def get_elite_trajectories(trajectories, q_param):
    total_rewards = [trajectory['total_reward'] for trajectory in trajectories]
    quantile = np.quantile(total_rewards, q=q_param) 
    return [trajectory for trajectory in trajectories if trajectory['total_reward'] > quantile]

In [9]:
env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5)
state_dim = 8
action_n = 4
lr = 1e-3

agent = CEM(state_dim, action_n, lr=lr)
episode_n = 50
trajectory_n = 100
trajectory_len = 500
q_param = 0.7

In [10]:
total_start = time.time()
for episode in range(episode_n):
    start = time.time()
    trajectories = [get_trajectory(env, agent, trajectory_len) for _ in range(trajectory_n)]
    
    mean_total_reward = np.mean([trajectory['total_reward'] for trajectory in trajectories])
    print(f'episode: {episode}, mean_total_reward = {mean_total_reward}')
    
    elite_trajectories = get_elite_trajectories(trajectories, q_param)
    
    if len(elite_trajectories) > 0:
        agent.update_policy(elite_trajectories)
    print('iteration_time: ', time.time() - start)
# get_trajectory(env, agent, trajectory_len, visualize=True)
print('total time', time.time() - total_start)

episode: 0, mean_total_reward = -199.46239613514362
iteration_time:  1.3514821529388428
episode: 1, mean_total_reward = -182.3854446109345
iteration_time:  1.4778614044189453
episode: 2, mean_total_reward = -176.7423894763417
iteration_time:  1.4938488006591797
episode: 3, mean_total_reward = -193.23823815642055
iteration_time:  1.3186006546020508
episode: 4, mean_total_reward = -167.40331441777212
iteration_time:  1.3194832801818848
episode: 5, mean_total_reward = -147.8692271190923
iteration_time:  1.32137131690979
episode: 6, mean_total_reward = -174.37895069458563
iteration_time:  1.3932733535766602
episode: 7, mean_total_reward = -177.9370494301973
iteration_time:  1.3604106903076172
episode: 8, mean_total_reward = -180.61101792144578
iteration_time:  1.3639898300170898
episode: 9, mean_total_reward = -161.11271475774768
iteration_time:  1.3589904308319092
episode: 10, mean_total_reward = -163.9249658056035
iteration_time:  1.3242990970611572
episode: 11, mean_total_reward = -164.